# Copy simulation results into the analysis repository
This notebook copies data from the simulation directory into the repository. If you have not run the simulations yourself you can skip this notebook and use the data in the repository instead. 

In [1]:
import os
import shutil
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

from PLBenchmarks import targets, ligands, edges
from tqdm.notebook import tqdm

import benchmarkpl
path = benchmarkpl.__path__[0]

{'mcl1_sample': {'name': 'mcl1_sample', 'date': datetime.date(2020, 8, 26), 'dir': '2020-08-26_mcl1_sample'}}


# Set path of simulation directory
Customize here if you have a specific simulation path

In [2]:
targets.set_data_dir('../../../02_benchmark_calculations/')

# Number of targets, ligands and edges in the data set

In [3]:
nligs, nedgs = 0, 0
print(f'{"Target":10s} {"Num Ligs":>10s} {"Num Edges":>10s}')
print(33 * '-')
for target in tqdm(targets.target_dict):
    print(f'{target:10s} {len(ligands.LigandSet(target)):10d} {len(edges.EdgeSet(target)):10d}')
    nligs += len(ligands.LigandSet(target))
    nedgs += len(edges.EdgeSet(target))
print(33 * '-')
print(f'{"total":10s} {nligs:10d} {nedgs:10d}')

Target       Num Ligs  Num Edges
---------------------------------


jnk1               21         31
pde2               21         34
thrombin           11         16
p38                34         56
ptp1b              23         49
galectin            8          7
cdk2               16         25
cmet               24         57
mcl1               42         71
bace               36         58
bace_hunt          32         60
bace_p2            12         26
tyk2               16         24
ros1               28         27
eg5                28         63
cdk8               33         54
hif2a              42         92
pfkfb3             40         66
pde10              35         36
shp2               26         56
syk                44         99
tnks2              27         60

---------------------------------
total             599       1067


# Copy data about targets, ligands, edges to repository
### Attention: Data is deleted before copying a new version. Only execute the cell if you know what you are doing.

In [10]:
target_yaml =  os.path.join(targets.data_path, 'targets.yml')
dest_path = os.path.join(path)
os.makedirs(dest_path, exist_ok=True)
if os.path.exists(target_yaml):
    shutil.copy(target_yaml,
                dest_path
               )
else:
    print(f'File {target_yaml} does not exist.')
for target in tqdm(targets.target_dict):
    dest_path = os.path.join(path, targets.get_target_dir(target), '00_data')
    data_path = os.path.join(targets.data_path, 
                             targets.get_target_dir(target), 
                             '00_data'
                            )
    if os.path.exists(data_path):
        if os.path.exists(dest_path):
            shutil.rmtree(dest_path)
        shutil.copytree(data_path,
                        dest_path
                       )
    else:
        print(f'Data path does not exist for target {target}')

# pmx calculations with openFF parameters (Hahn et al.)
Copies raw result files from calculations into repository path

In [11]:
# function to retrieve data from PLBenchmarks calculations
def copyRawResults(target, forcefield='openff-1.0.0.offxml'):
    # copy raw dat file
    file_path = os.path.join(targets.data_path, 
                             targets.get_target_dir(target), 
                             '06_pmx',
                             f'{forcefield}',
                             'results', 
                             f'{target}_{forcefield}.dat'
                            )
    if not os.path.exists(file_path):
        print(f'File {file_path} does not exist.')
    else:
        # copy file to backup directory
        shutil.copy(file_path, 
                    os.path.join(path, '..', '00_data', 'input')
                   )
    # copy csv file with more detailed information
    file_path = os.path.join(targets.data_path, 
                             targets.get_target_dir(target), 
                             '06_pmx',
                             f'{forcefield}',
                             'results', 
                             f'{target}_{forcefield}.csv'
                            )
    if not os.path.exists(file_path):
        print(f'File {file_path} does not exist.')
    else:
        # copy file to backup directory
        shutil.copy(file_path, 
                    os.path.join(path, '..', '00_data', 'input')
                   )

In [12]:
# choose forcefields here
forcefields = ['openff-1.0.0.offxml']

In [13]:
for target in tqdm(targets.target_dict):
    for forcefield in forcefields:
        copyRawResults(target, forcefield)